<a href="https://colab.research.google.com/github/santos-luciano/reddit_collect/blob/main/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando a biblioteca praw

In [1]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 4.7 MB/s eta 0:00:00


# Lendo as bibliotecas necessárias

In [ ]:
import praw
import pandas as pd
from transformers import pipeline



# Carregando comentários de um post

In [ ]:
reddit = praw.Reddit(
    client_id="5gzN-N3yiwaawbtRDTniCw",
    client_secret="gQzhykzk0MQ4yiWaYJrrpGRf-fUgpw",
    user_agent="meu_script:v1.0 (by u/LegFantastic551)"
)

# Post no Reddit 1b50d6q

submission = reddit.submission(id="1nvr6et")
submission.comments.replace_more(limit=0)

data = []
for comment in submission.comments.list():
    data.append({
        "post_title": submission.title,
        "comment_body": comment.body,
        "comment_score": comment.score,
        "comment_author": str(comment.author)
    })

df = pd.DataFrame(data)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
df.head()

,post_title,comment_body,comment_score,comment_author
0,"Por unanimidade, Câmara aprova isenção do IR p...","FOI UNANIMIDADE, HISTÓRICO!\n\nParabéns ao pro...",246,Razz_SeriousSloth
1,"Por unanimidade, Câmara aprova isenção do IR p...",O que um cagaço de ver milhares de pessoas nas...,338,Bananey
2,"Por unanimidade, Câmara aprova isenção do IR p...","A parte mais histórica: ""O aumento da faixa de...",190,oAbuttre
3,"Por unanimidade, Câmara aprova isenção do IR p...",Acho que precisamos ir mais as ruas.\n\nA prop...,95,SCLST_F_Hell
4,"Por unanimidade, Câmara aprova isenção do IR p...","Esperava que fosse aprovado, mas por unanimida...",30,BShep_OLDBSN


In [ ]:
# Carrega modelo

classifier = pipeline("sentiment-analysis",
                      model="nlptown/bert-base-multilingual-uncased-sentiment",
                      truncation = True,
                      max_length = 512)

political_negatives = ["superfaturada", "corrupção", "ineficiente", "vergonha", "fracasso"]

def sentiment_custom(comment):
    # Prioriza termos políticos
    if any(word in comment.lower() for word in political_negatives):
        return "negativo"
    else:
        # Usa modelo BERT para o resto
        res = classifier(comment)[0]
        stars = int(res['label'][0])
        if stars <= 2:
            return "negativo"
        elif stars == 3:
            return "neutro"
        else:
            return "positivo"

# Classificar cada comentário
resultados = [sentiment_custom(c) for c in df.comment_body]

Device set to use cpu


In [ ]:
resultados

['negativo',
 'negativo',
 'neutro',
 'negativo',
 'negativo',
 'positivo',
 'negativo',
 'positivo',
 'positivo']

In [ ]:
# Mostrar resultados
for c, r in zip(df.comment_body[1:10], resultados):
    print(f"{c} -> {r}")


O que um cagaço de ver milhares de pessoas nas ruas não faz com esse Congresso de merda -> negativo
A parte mais histórica: "O aumento da faixa de isenção será compensado taxando mais quem ganha acima de R$ 600 mil por ano."

E imaginar que essa tabela estava congelada desde 2017, e agora tem sido reajustada TODO ano desde que o Lula entrou... Só isso já garante a reeleição... -> negativo
Acho que precisamos ir mais as ruas.

A propósito, conterrâneos de SP, Sabesp tá uma bosta privatizada, só pra lembrar… -> neutro
Esperava que fosse aprovado, mas por unanimidade e sem mexeram na taxação dos ricos que ganham acima de 600 mil reais e retenção na fonte sobre lucros e dividendos acima de 50 mil? Isso eu não esperava. 😲

Foi uma vitória esmagadora do governo Lula e do povo.  
Mobilização das esquerdas e progressistas nas ruas surte efeito. ❤️❤️

Quero ver agora algum Zé Ruela vir dizer o contrário agora. -> negativo
Quem teve mais tempo para entender eu fiquei em dúvida. 

Para pessoas fí

In [ ]:
df.comment_body[1:].str.cat(sep=' ')

'O serviço péssimo do ferry parece até lobby pra ponte. Todo mundo quer Espero seja concluída. Porque vai ficar bonito vê ela sobre essas águas. Eu soube que vai inaugurar em 2014 Dizem que isso ia acabar com a ilha. Ia virar mais um bairro de Salvador. Tendo a acreditar. Mas ajudaria muita gente também. Morei lá um tempo atrás. Penso que a ponte transformaria Itaparica em uma Niterói. Se com o metro já deu para desviar imagina com a ponte. Eu era criança tavam dizendo que essa pronta logo logo ficava pronta \nJá sou adulto Acho massa, mas queria que os chineses metessem uns trem de alta velocidade aqui logo, ao invés dos carros elétricos\n\nDeixa os EUA se lascar pra lá com modal rodoviário Uma coisa que pouco se discute é o impacto ambiental que vai gerar durante a construção da ponte, acho que o transporte aquático de qualidade seria mais viável Metrô 2 eu ja morei na ilha e eu posso dizer com absoluta certeza (e tristeza) que nenhum dos municípios está pronto para o fluxo que está 

In [ ]:
# Instalar se ainda não tiver
# pip install wordcloud matplotlib nltk

from wordcloud import WordCloud
import matplotlib.pyplot as plt+
from nltk.corpus import stopwords
import nltk

# Baixar stopwords em português
nltk.download('stopwords')

# Texto de exemplo
texto = df.comment_body.str.cat(sep=' ')

# Lista de stopwords em português
stop_words = set(stopwords.words('portuguese'))

# Gerar nuvem de palavras
nuvem = WordCloud(width=800, height=400,
                  background_color='white',
                  stopwords=stop_words,
                  collocations=False).generate(texto)

# Mostrar resultado
plt.figure(figsize=(10,5))
plt.imshow(nuvem, interpolation='bilinear')
plt.axis("off")
plt.show()


SyntaxError: invalid syntax (ipython-input-3276925785.py, line 5)

,post_title,comment_body,comment_score,comment_author,sentiment,sentiment_label,sentiment_tb,sentiment_label_tb
0,No concurso de ideia mais burra para resolver ...,"""O Novo é diferente, é top, é moderno.""",226,Tonho_O_Faxineiro,0.2023,positivo,0.500000,positivo
1,No concurso de ideia mais burra para resolver ...,Zema é 100% de aproveitamento zero.,158,AlbatrossOk6223,0.0000,neutro,0.000000,neutro
2,No concurso de ideia mais burra para resolver ...,Sair do BRICS é falir a bancada do agronegócio...,41,CrisBrl93,0.0000,neutro,0.000000,neutro
3,No concurso de ideia mais burra para resolver ...,[deleted],138,None,0.0000,neutro,0.000000,neutro
4,No concurso de ideia mais burra para resolver ...,E ainda tem um monte de desavisado que vai vot...,17,Tall-Series-1091,-0.5267,negativo,0.000000,neutro
...,...,...,...,...,...,...,...,...
90,No concurso de ideia mais burra para resolver ...,"Cara, não faço ideia. Mas a gente sabe informa...",7,AppropriateBreak1076,-0.5267,negativo,0.000000,neutro
91,No concurso de ideia mais burra para resolver ...,"sim, muito pior mesmo. Você está super fragili...",2,fuckyesnewuser,0.6360,positivo,0.416667,positivo
92,No concurso de ideia mais burra para resolver ...,As vezes o mano virou comunista e você pode re...,3,GrievousBR,0.0000,neutro,0.000000,neutro
93,No concurso de ideia mais burra para resolver ...,"Rsrsrs pior que as aulas dele eram massa, só s...",3,AppropriateBreak1076,-0.2960,negativo,0.000000,neutro
